<a href="https://colab.research.google.com/github/yamada2310/dataAnalysis100/blob/main/dataAnalysis100_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/MyDrive/100knock-data_analytics/5章')

In [3]:
import pandas as pd
customer = pd.read_csv('customer_join.csv')
uselog_months = pd.read_csv('use_log_months.csv')

In [4]:
year_months = list(uselog_months["年月"].unique())
uselog = pd.DataFrame()
for i in range(1, len(year_months)):
  tmp = uselog_months.loc[uselog_months["年月"]==year_months[i]].copy()
  tmp.rename(columns={"count":"count_0"},inplace=True)
  tmp_before = uselog_months.loc[uselog_months["年月"]==year_months[i-1]].copy()
  del tmp_before["年月"]
  tmp_before.rename(columns={"count":"count_1"},inplace=True)
  tmp = pd.merge(tmp,tmp_before, on="customer_id",how="left")
  uselog=pd.concat([uselog,tmp],ignore_index=True)
uselog.head()

,年月,customer_id,count_0,count_1
0,201805,AS002855,5,4.0
1,201805,AS009373,4,3.0
2,201805,AS015233,7,NaN
3,201805,AS015315,3,6.0
4,201805,AS015739,5,7.0


In [7]:
from dateutil.relativedelta import relativedelta
exit_customer = customer.loc[customer["is_deleted"]==1].copy()
exit_customer["exit_date"]=None
exit_customer["end_date"]=pd.to_datetime(exit_customer["end_date"])
for i in exit_customer.index:
  exit_customer.loc[i,"exit_date"]=exit_customer.loc[i,"end_date"]-relativedelta(months=1)
exit_customer["exit_date"]=pd.to_datetime(exit_customer["exit_date"])
exit_customer["年月"]=exit_customer["exit_date"].dt.strftime("%Y%m")
uselog["年月"] = uselog["年月"].astype(str)
exit_uselog=pd.merge(uselog,exit_customer,on=["customer_id","年月"],how="left")
print(len(uselog))
exit_uselog.head()

33851


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201805,AS002855,5,4.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,201805,AS009373,4,3.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2,201805,AS015233,7,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3,201805,AS015315,3,6.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4,201805,AS015739,5,7.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [8]:
exit_uselog=exit_uselog.dropna(subset=["name"])
print(len(exit_uselog))
print(len(exit_uselog["customer_id"].unique()))
exit_uselog.head()

1104
1104


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
19,201805,AS055680,3,3.0,XXXXX,C01,M,2018-03-01,2018-06-30,CA1,...,10500.0,通常,3.000000,3.0,3.0,3.0,0.0,2018-06-30,3.0,2018-05-30
57,201805,AS169823,2,3.0,XX,C01,M,2017-11-01,2018-06-30,CA1,...,10500.0,通常,3.000000,3.0,4.0,2.0,1.0,2018-06-30,7.0,2018-05-30
110,201805,AS305860,5,3.0,XXXX,C01,M,2017-06-01,2018-06-30,CA1,...,10500.0,通常,3.333333,3.0,5.0,2.0,0.0,2018-06-30,12.0,2018-05-30
128,201805,AS363699,5,3.0,XXXXX,C01,M,2018-02-01,2018-06-30,CA1,...,10500.0,通常,3.333333,3.0,5.0,2.0,0.0,2018-06-30,4.0,2018-05-30
147,201805,AS417696,1,4.0,XX,C03,F,2017-09-01,2018-06-30,CA1,...,6000.0,通常,2.000000,1.0,4.0,1.0,0.0,2018-06-30,9.0,2018-05-30


In [9]:
conti_customer=customer.loc[customer["is_deleted"]==0]
conti_uselog=pd.merge(uselog,conti_customer,on=["customer_id"],how="left")
print(len(conti_uselog))
conti_uselog=conti_uselog.dropna(subset=["name"])
print(len(conti_uselog))

33851
27422


In [12]:
conti_uselog=conti_uselog.sample(frac=1,random_state=0).reset_index(drop=True)
conti_uselog=conti_uselog.drop_duplicates(subset="customer_id")
print(len(conti_uselog))
conti_uselog.head()

2842


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,201810,OA245705,7,9.0,XXXXX,C02,F,2018-07-09,NaN,CA1,...,デイタイム,7500.0,通常,7.222222,7.0,9.0,6.0,1.0,2019-04-30,9.0
1,201808,PL270116,5,4.0,XXXXX,C01,M,2015-05-01,NaN,CA1,...,オールタイム,10500.0,通常,5.083333,5.0,7.0,3.0,1.0,2019-04-30,47.0
2,201806,PL834411,2,3.0,XXXXX,C03,F,2015-05-01,NaN,CA1,...,ナイト,6000.0,通常,4.916667,4.5,8.0,1.0,1.0,2019-04-30,47.0
3,201809,GD022268,2,5.0,XXX,C02,M,2015-05-01,NaN,CA1,...,デイタイム,7500.0,通常,4.166667,5.0,7.0,1.0,1.0,2019-04-30,47.0
4,201805,OA544396,1,5.0,XXXXX,C03,F,2015-09-01,NaN,CA1,...,ナイト,6000.0,通常,4.750000,5.0,7.0,1.0,1.0,2019-04-30,43.0


In [13]:
predict_data = pd.concat([conti_uselog,exit_uselog],ignore_index=True)
print(len(predict_data))
predict_data.head()

3946


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201810,OA245705,7,9.0,XXXXX,C02,F,2018-07-09,NaT,CA1,...,7500.0,通常,7.222222,7.0,9.0,6.0,1.0,2019-04-30,9.0,NaT
1,201808,PL270116,5,4.0,XXXXX,C01,M,2015-05-01,NaT,CA1,...,10500.0,通常,5.083333,5.0,7.0,3.0,1.0,2019-04-30,47.0,NaT
2,201806,PL834411,2,3.0,XXXXX,C03,F,2015-05-01,NaT,CA1,...,6000.0,通常,4.916667,4.5,8.0,1.0,1.0,2019-04-30,47.0,NaT
3,201809,GD022268,2,5.0,XXX,C02,M,2015-05-01,NaT,CA1,...,7500.0,通常,4.166667,5.0,7.0,1.0,1.0,2019-04-30,47.0,NaT
4,201805,OA544396,1,5.0,XXXXX,C03,F,2015-09-01,NaT,CA1,...,6000.0,通常,4.750000,5.0,7.0,1.0,1.0,2019-04-30,43.0,NaT


In [14]:
predict_data["period"]=0
predict_data["now_date"]=pd.to_datetime(predict_data["年月"],format="%Y%m")
predict_data["start_date"]=pd.to_datetime(predict_data["start_date"])
for i in range(len(predict_data)):
  delta=relativedelta(predict_data.loc[i,"now_date"],predict_data.loc[i,"start_date"])
  predict_data.loc[i,"period"]=int(delta.years*12+delta.months)
predict_data.head()

,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date,period,now_date
0,201810,OA245705,7,9.0,XXXXX,C02,F,2018-07-09,NaT,CA1,...,7.222222,7.0,9.0,6.0,1.0,2019-04-30,9.0,NaT,2,2018-10-01
1,201808,PL270116,5,4.0,XXXXX,C01,M,2015-05-01,NaT,CA1,...,5.083333,5.0,7.0,3.0,1.0,2019-04-30,47.0,NaT,39,2018-08-01
2,201806,PL834411,2,3.0,XXXXX,C03,F,2015-05-01,NaT,CA1,...,4.916667,4.5,8.0,1.0,1.0,2019-04-30,47.0,NaT,37,2018-06-01
3,201809,GD022268,2,5.0,XXX,C02,M,2015-05-01,NaT,CA1,...,4.166667,5.0,7.0,1.0,1.0,2019-04-30,47.0,NaT,40,2018-09-01
4,201805,OA544396,1,5.0,XXXXX,C03,F,2015-09-01,NaT,CA1,...,4.750000,5.0,7.0,1.0,1.0,2019-04-30,43.0,NaT,32,2018-05-01


In [15]:
predict_data.isna().sum()

年月                      0
customer_id             0
count_0                 0
count_1               254
name                    0
class                   0
gender                  0
start_date              0
end_date             2842
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2842
period                  0
now_date                0
dtype: int64

In [16]:
predict_data=predict_data.dropna(subset=["count_1"])
predict_data.isna().sum()

年月                      0
customer_id             0
count_0                 0
count_1                 0
name                    0
class                   0
gender                  0
start_date              0
end_date             2640
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2640
period                  0
now_date                0
dtype: int64